In [1]:
import os
%%time

from adlfs import AzureBlobFileSystem
from common.functions.functions import Functions
import time

import json
import pandas
import pandas as pd
from tqdm import tqdm

from common.schemas.pyarrow_schema import tagging_schema
from common.storage.azure_file_storage import AzureFileStorageAdapter
from common.captioning.azure_descriptions import AzureCaption
from common.schemas.pyarrow_schema import schema
from PIL import Image
import requests

tqdm.pandas(desc="Progress")
file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

functions: Functions = Functions()

caption: AzureCaption = AzureCaption(file_system)

CPU times: total: 5.34 s
Wall time: 3min 1s


In [12]:
df: pandas.DataFrame = pandas.read_parquet('data/parquet/thumbnail_curation.parquet', engine='pyarrow',
										   filesystem=file_system)

sexy = df.where(
	(df['model'] == 'SexyDiffusion') |
	(df['model'] == 'SWFPetite') |
	(df['model'] == 'RedHeadDiffusion') |
	(df['model'] == 'NextDoorGirlsDiffusion') |
	(df['model'] == 'SexyAsianDiffusion') |
	(df['model'] == 'PrettyGirlDiffusion'))

sexy = sexy.where(df['thumbnail_accept'] == True)

sexy.dropna(inplace=True)
sexy.reset_index(inplace=True, drop=True)

display(sexy)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person, human face, clothing, lady, smile, ch...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person, human face, skin, eyelash, eyebrow, b...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing, person, human face, smile, shoulder...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person, human face, indoor, woman, chest, sho...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person, human face, smile, clothing, woman, i...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,10sx708,SFWRedheads,Delilaylayx,Lets go out for diner?,a woman in a blue braille braille braille brai...,616f67018f3105512aec4c20fcd34edb,/r/SFWRedheads/comments/10sx708/lets_go_out_fo...,https://i.redd.it/0whemz6nb3ga1.jpg,10sx708.jpg,data/image/10sx708.jpg,...,True,True,True,[],a woman in a blue lingerie,data/image/thumbnail/10sx708.jpg,True,True,True,"[a woman in a blue lingerie, a woman in a blue..."
1354,10sybk7,SFWRedheads,SophiasSelfies,Beach day 😁💕,a woman in a yellow bikini top and a yellow bi...,72ebde1b0cebc8f9b00d9c2a82043d07,/r/SFWRedheads/comments/10sybk7/beach_day/,https://i.redd.it/06mxlz9wj3ga1.jpg,10sybk7.jpg,data/image/10sybk7.jpg,...,True,True,True,[],a woman in a garment on a beach,data/image/thumbnail/10sybk7.jpg,True,True,True,"[a woman in a garment on a beach, a woman taki..."
1355,10t11nz,SFWRedheads,Wallydinger123,Lunch date,a woman sitting at a table with a menu in fron...,0abf63132d8fafe5c1c6bbc52b9875b6,/r/SFWRedheads/comments/10t11nz/lunch_date/,https://i.imgur.com/pmiQeDA.jpg,10t11nz.jpg,data/image/10t11nz.jpg,...,True,True,True,[],a woman sitting at a table with menus and a wi...,data/image/thumbnail/10t11nz.jpg,True,True,True,"[a woman taking a selfie, a woman sitting at a..."
1356,10te4ym,SFWRedheads,aurelie_027,is it too early to wish good morning?,a woman with a black bra top and bra top bra t...,571edabdb7322a69325b03246de32cd1,/r/SFWRedheads/comment

In [ ]:
os.makedirs('data', exist_ok=True)

In [23]:
lines = []
for elem in sexy.to_dict(orient='records'):
	image_name = elem['image_name']
	captions = elem['additional_captions'].tolist()
	captions.append(elem['azure_caption'])
	captions.append(elem['caption'])
	captions.append(elem['title'])
	captions = list(set(captions))
	data_row = {'file_name': image_name, 'text': captions}
	lines.append(data_row)
	file_system.cp( f'data/image/thumbnail/{image_name}', 'data/training/sexy')

df = pandas.DataFrame(data=lines)
display(df)

,file_name,text
0,1013bdt.jpg,"[a close up of an eye, arafed image of a woman..."
1,105mekt.jpg,[a close up of a woman with red hair and a whi...
2,105qvgl.jpg,"[a close-up of a woman's face, a person wearin..."
3,105rpcj.jpg,[a woman in a blue bikini top and a blue bra t...
4,105styc.jpg,"[Low key… still bangable?, a close up of a per..."
...,...,...
1353,10sx708.jpg,"[a woman in a blue garment, a woman in a blue ..."
1354,10sybk7.jpg,"[Beach day 😁💕, a woman wearing a yellow garmen..."
1355,10t11nz.jpg,[a woman sitting at a table with menus and a w...
1356,10te4ym.jpg,"[close up of a person's face, a woman with a b..."


In [25]:
df.to_json('metadata.jsonl', orient='records', lines=True)

file_system.upload('metadata.jsonl', 'data/training/sexy')

[None]

In [31]:
gpt_lines = []
for record in sexy.to_dict(orient='records'):
	name = record['model']
	prompt = record['title']
	captions = record['additional_captions'].tolist()
	captions.append(record['azure_caption'])
	captions.append(record['caption'])
	captions.append(record['title'])
	captions = list(set(captions))
	for caption in captions:
		line = f"<|startoftext|><|model|>SexyGirl<|prompt|>{prompt}<|text|>{caption}<|endoftext|>" + "\n"
		gpt_lines.append(line)

with open("training.txt", "wb") as f:
	for line in gpt_lines:
		f.write(line.encode("utf-8"))

In [33]:
file_system.upload('training.txt', 'data/training/sexy_gpt.txt', overwrite=True)

[None]

In [34]:
file_system.ls('data/training/sexy_gpt.txt')

['data/training/sexy_gpt.txt']

In [40]:
file_system.upload('sexy', 'data/sexy', recursive=True)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,